In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('dataset\Bitext_Sample_Customer_Service_Training_Dataset.csv')

In [4]:
#full brittext data
df2 = pd.read_csv('dataset\\big_set\\big_set.csv')

In [ ]:
#twitter customer service dataset

In [76]:
df2.head()

,flags,utterance,category,intent
0,BILC,"I don't have an online account, what do I have...",ACCOUNT,create_account
1,BILQZ,can you tell me if i can regisger two accounts...,ACCOUNT,create_account
2,BPLC,"I have no online account, open one, please",ACCOUNT,create_account
3,BIPLD,"could you ask an agent how to open an account,...",ACCOUNT,create_account
4,BLQC,"i want an online account, create one",ACCOUNT,create_account


In [5]:
df2.shape

(21534, 4)

In [78]:
df2.isnull().sum()

flags        0
utterance    0
category     0
intent       0
dtype: int64

In [79]:
blanks = []  # start with an empty list

for i,f,utt,cat,intent in df2.itertuples():  # iterate over the DataFrame
    if type(utt)==str:            # avoid NaN values
        if utt.isspace():         # test 'review' for whitespace
            blanks.append(i)     # add matching index numbers to the list
        
print(len(blanks), 'blanks: ', blanks)

0 blanks:  []


**Could implement other methods here such as lemmatization as further input processing.**

In [81]:
#split data ensuring all intents in both train + test sets
from sklearn.model_selection import train_test_split
X = df2['utterance']
y = df2['category']
X_train, X_test, y_train, y_test = train_test_split(X,y, stratify= df2[['category']], test_size=0.33, random_state=42)

In [82]:
#pipeline tfidf and model
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

intent_cat_lsvc = Pipeline([('tfidf', TfidfVectorizer()),('clf', LinearSVC())])

In [83]:
intent_cat_lsvc.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [84]:
predictions = intent_cat_lsvc.predict(X_test)

In [85]:
# Report the confusion matrix
from sklearn import metrics
print(metrics.confusion_matrix(y_test,predictions))

[[1503    0    0    0    0    0    0    0    1    0    0]
 [   0  119    0    0    0    0    0    0    0    0    0]
 [   1    0 1016    0    0    0    0    0    0    0    0]
 [   0    0    0  165    0    0    0    0    0    0    0]
 [   0    0    0    0  438    0    0    0    0    0    0]
 [   0    0    0    0    0  804    0    2    0    0    0]
 [   0    0    0    0    0    0   78    0    0    0    0]
 [   0    0    0    1    0    0    0  743    0    0    0]
 [   0    0    0    0    0    1    0    0 1529    0    0]
 [   0    0    0    0    1    1    0    3    0  633    0]
 [   0    0    0    0    0    0    0    0    0    0   68]]


In [86]:
print(metrics.classification_report(y_test,predictions))

                  precision    recall  f1-score   support

         ACCOUNT       1.00      1.00      1.00      1504
CANCELLATION_FEE       1.00      1.00      1.00       119
         CONTACT       1.00      1.00      1.00      1017
        DELIVERY       0.99      1.00      1.00       165
        FEEDBACK       1.00      1.00      1.00       438
        INVOICES       1.00      1.00      1.00       806
      NEWSLETTER       1.00      1.00      1.00        78
           ORDER       0.99      1.00      1.00       744
         PAYMENT       1.00      1.00      1.00      1530
         REFUNDS       1.00      0.99      1.00       638
        SHIPPING       1.00      1.00      1.00        68

        accuracy                           1.00      7107
       macro avg       1.00      1.00      1.00      7107
    weighted avg       1.00      1.00      1.00      7107



In [106]:
intent_cat_lsvc.predict([""])

array(['SHIPPING'], dtype=object)